In [1]:
# Eval code, adapted for jupyter notebook with backtest decorator
import numpy as np
import pandas as pd
from functools import wraps

nInst = 0
nt = 0
commRate = 0.0005
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep=r'\s+', header=None, index_col=None)
    (nt,nInst) = df.shape
    return (df.values).T

pricesFile="./.data/2025/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(getPosition, prcHist=None, numTestDays=200):
    if prcHist is None:
        prcHist = prcAll
    
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt_local) = prcHist.shape
    startDay = nt_local + 1 - numTestDays
    
    for t in range(startDay, nt_local+1):
        prcHistSoFar = prcHist[:,:t]
        curPrices = prcHistSoFar[:,-1]
        if (t < nt_local):
            # Trading, do not do it on the very last day of the test
            newPosOrig = getPosition(prcHistSoFar)
            posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
            newPos = np.clip(newPosOrig, -posLimits, posLimits)
            deltaPos = newPos - curPos
            dvolumes = curPrices * np.abs(deltaPos)
            dvolume = np.sum(dvolumes)
            totDVolume += dvolume
            comm = dvolume * commRate
            cash -= curPrices.dot(deltaPos) + comm
        else:
            newPos = np.array(curPos)
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        if (t > startDay):
            print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
            todayPLL.append(todayPL)
    
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(249) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

def backtest(numTestDays=200, prcHist=None, verbose=True):
    """
    Decorator to backtest a position function
    
    Args:
        numTestDays: Number of days to test (default 200)
        prcHist: Price history array (default uses global prcAll)
        verbose: Whether to print detailed results (default True)
    """
    def decorator(getPosition_func):
        @wraps(getPosition_func)
        def wrapper(*args, **kwargs):
            print(f"Testing function: {getPosition_func.__name__}")
            
            # Run the backtest
            (meanpl, ret, plstd, sharpe, dvol) = calcPL(
                getPosition_func, 
                prcHist if prcHist is not None else prcAll, 
                numTestDays
            )
            
            score = meanpl - 0.1*plstd
            
            if verbose:
                print ("=====")
                print (f"Function: {getPosition_func.__name__}")
                print ("mean(PL): %.1lf" % meanpl)
                print ("return: %.5lf" % ret)
                print ("StdDev(PL): %.2lf" % plstd)
                print ("annSharpe(PL): %.2lf " % sharpe)
                print ("totDvolume: %.0lf " % dvol)
                print ("Score: %.2lf" % score)
                print ("=====")
            
            # Store results as function attributes for later access
            wrapper.results = {
                'meanpl': meanpl,
                'return': ret,
                'plstd': plstd,
                'sharpe': sharpe,
                'dvol': dvol,
                'score': score
            }
            
            return wrapper.results
        
        # Run backtest immediately when decorator is applied
        wrapper()
        return wrapper
    
    return decorator



Loaded 50 instruments for 750 days


In [3]:
@backtest()
def simple_long_position(prcHistSoFar):
    """Simple strategy: always long 1 share of each stock"""
    nInst = prcHistSoFar.shape[0]
    return np.ones(nInst)

Testing function: simple_long_position
Day 552 value: -4.31 todayPL: $-3.08 $-traded: 2462 return: -0.00175
Day 553 value: -5.12 todayPL: $-0.81 $-traded: 2462 return: -0.00208
Day 554 value: -8.58 todayPL: $-3.46 $-traded: 2462 return: -0.00349
Day 555 value: -5.13 todayPL: $3.45 $-traded: 2462 return: -0.00208
Day 556 value: -5.90 todayPL: $-0.77 $-traded: 2462 return: -0.00240
Day 557 value: -16.58 todayPL: $-10.68 $-traded: 2462 return: -0.00674
Day 558 value: -19.75 todayPL: $-3.17 $-traded: 2462 return: -0.00802
Day 559 value: -18.38 todayPL: $1.37 $-traded: 2462 return: -0.00747
Day 560 value: -26.68 todayPL: $-8.30 $-traded: 2462 return: -0.01084
Day 561 value: -30.60 todayPL: $-3.92 $-traded: 2462 return: -0.01243
Day 562 value: -31.75 todayPL: $-1.15 $-traded: 2462 return: -0.01290
Day 563 value: -32.52 todayPL: $-0.77 $-traded: 2462 return: -0.01321
Day 564 value: -36.40 todayPL: $-3.88 $-traded: 2462 return: -0.01479
Day 565 value: -36.23 todayPL: $0.17 $-traded: 2462 retur

In [ ]:
@backtest()
def momentum_strategy(prcHistSoFar):
    """Simple momentum strategy"""
    nInst, nt = prcHistSoFar.shape
    if nt < 2:
        return np.zeros(nInst)
    
    # Calculate recent returns
    recent_returns = (prcHistSoFar[:, -1] - prcHistSoFar[:, -2]) / prcHistSoFar[:, -2]
    
    # Go long on positive momentum, short on negative
    positions = np.sign(recent_returns)
    return positions


In [12]:
@backtest()
def mean_reversion_strategy(prcHistSoFar):
    """Simple mean reversion strategy"""
    nInst, nt = prcHistSoFar.shape
    if nt < 10:
        return np.zeros(nInst)
    
    # Calculate deviation from 10-day mean
    recent_mean = np.mean(prcHistSoFar[:, -10:], axis=1)
    current_price = prcHistSoFar[:, -1]
    deviation = (current_price - recent_mean) / recent_mean
    
    # Go opposite to deviation (mean reversion)
    positions = -np.sign(deviation)
    return positions

Testing function: mean_reversion_strategy
Day 552 value: 1.18 todayPL: $2.41 $-traded: 3522 return: 0.00033
Day 553 value: -1.76 todayPL: $-2.94 $-traded: 3899 return: -0.00045
Day 554 value: 0.52 todayPL: $2.28 $-traded: 4934 return: 0.00011
Day 555 value: -0.34 todayPL: $-0.86 $-traded: 5520 return: -0.00006
Day 556 value: 1.08 todayPL: $1.42 $-traded: 6741 return: 0.00016
Day 557 value: 4.61 todayPL: $3.53 $-traded: 8202 return: 0.00056
Day 558 value: 3.92 todayPL: $-0.68 $-traded: 9031 return: 0.00043
Day 559 value: 7.05 todayPL: $3.12 $-traded: 10290 return: 0.00068
Day 560 value: 9.94 todayPL: $2.90 $-traded: 11572 return: 0.00086
Day 561 value: 6.74 todayPL: $-3.20 $-traded: 12342 return: 0.00055
Day 562 value: 7.17 todayPL: $0.44 $-traded: 13291 return: 0.00054
Day 563 value: 9.84 todayPL: $2.66 $-traded: 14502 return: 0.00068
Day 564 value: 13.21 todayPL: $3.37 $-traded: 16032 return: 0.00082
Day 565 value: 11.50 todayPL: $-1.72 $-traded: 17288 return: 0.00066
Day 566 value: 9

In [4]:
@backtest(10)
def getPosition(prices):
    return [1] * len(prices)

Testing function: getPosition
Day 742 value: -3.95 todayPL: $-2.74 $-traded: 2413 return: -0.00164
Day 743 value: -2.80 todayPL: $1.15 $-traded: 2413 return: -0.00116
Day 744 value: -2.82 todayPL: $-0.02 $-traded: 2413 return: -0.00117
Day 745 value: -6.27 todayPL: $-3.45 $-traded: 2413 return: -0.00260
Day 746 value: -8.18 todayPL: $-1.91 $-traded: 2413 return: -0.00339
Day 747 value: -11.16 todayPL: $-2.98 $-traded: 2413 return: -0.00462
Day 748 value: -8.86 todayPL: $2.30 $-traded: 2413 return: -0.00367
Day 749 value: -10.58 todayPL: $-1.72 $-traded: 2413 return: -0.00438
Day 750 value: -9.49 todayPL: $1.09 $-traded: 2413 return: -0.00393
=====
Function: getPosition
mean(PL): -0.9
return: -0.00393
StdDev(PL): 1.97
annSharpe(PL): -7.35 
totDvolume: 2413 
Score: -1.12
=====


In [ ]:
# A bunch of basic algos that can be used as samples:
# 1: If trend continues for n consecutive days, position moves by n steps